# Spam Classification

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

## Read the Data

In [2]:
data = pd.read_csv('./data/spam_data.csv')
data.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


## Text Preprocessing

In [3]:
import re # regex library
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

## Train, Test Split

In [4]:
from sklearn.model_selection import train_test_split
X = data['Message']
y = data['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Training a Neural Network Pipeline

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [6]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, max_features=700, preprocessor=preprocessor, ngram_range=(1,1))
neural_net_pipeline = Pipeline([('vectorizer', tfidf), ('nn', MLPClassifier(hidden_layer_sizes=(700, 700)))])

In [7]:
neural_net_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(lowercase=False, max_features=700,
                                 preprocessor=<function preprocessor at 0x7fa237536a70>)),
                ('nn', MLPClassifier(hidden_layer_sizes=(700, 700)))])

## Testing the Pipeline

In [8]:
y_pred = neural_net_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1448
        spam       0.97      0.92      0.94       224

    accuracy                           0.99      1672
   macro avg       0.98      0.96      0.97      1672
weighted avg       0.98      0.99      0.98      1672

Accuracy: 98.50478468899522 %


## Saving the Pipeline

In [9]:
from joblib import dump
dump(neural_net_pipeline, 'spam_classifier.joblib')

['spam_classifier.joblib']